In [1]:
# import packages

In [2]:
from pandas.io.json import json_normalize
import json
from pandasql import sqldf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
import io
import requests
import json
import base64
import datetime
from urllib.parse import urlencode

In [3]:
# client id

In [4]:
client_id = '6e24789ec7874259a220f747fcd7e93e'
client_secret = '9bd8ff1ad2f34ef6a87f00124be070b2'

In [5]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"



    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret

    def getClientCreds(self):
        '''Returns b64 encoded string'''
        client_id = self.client_id
        client_secret = self.client_secret

        if client_id == None or client_secret == None:
            raise Exception('Must set a client id and secret')

        client_creds  = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()

    def getTokenHeader(self):
        client_creds_b64 = self.getClientCreds()
        return {
            'Authorization':f"Basic {client_creds_b64}"
        }

    def getTokenData(self):
        return {
            "grant_type":"client_credentials"
        }

    def perform_auth(self):
        token_url = self.token_url
        token_data = self.getTokenData()
        token_header = self.getTokenHeader()

        r = requests.post(token_url, data=token_data, headers=token_header)

        if r.status_code  not in range(200,299):
            raise Exception('Could not authenticate client.')
            #return False

        now = datetime.datetime.now()
        token_response_data = r.json()
        access_token = token_response_data['access_token']
        expires_in = token_response_data['expires_in']
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now


        return True

    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now= datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token() 
        return token
    
    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {
            "Authorization": f"Bearer {access_token}", 
        }
        return headers
    
    def get_resource(self, lookup_id, resource_type='albums', version='v1'):
        endpoint = f'https://api.spotify.com/{version}/{resource_type}/{lookup_id}'
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers = headers)
        if r.status_code not in range (200, 299):
            return {}
        return r.json()
    
    def get_album(self, _id ):
        return self.get_resource(_id, resource_type='albums')
    
    def get_artist(self, _id ):
        return self.get_resource(_id, resource_type='artists')
    
    def get_playlist(self, _id):
        return self.get_resource(_id, resource_type='playlists')
    
    
    def base_search(self, query_params): #type
        access_token = self.get_access_token()
        headers = self.get_resource_header()
        endpoint = "https://api.spotify.com/v1/search"
        lookup_URL = f"{endpoint}?{query_params}"
        r = requests.get(lookup_URL, headers=headers)
        if r.status_code not in range (200, 299):
            return {}
        return r.json()
    
    def search(self, query=None, operator=None, operator_query=None, search_type='artist'):
        if query == None:
            raise Exception('A query is required')
        if isinstance(query, dict):
            query = ' '.join([f'{k}:{v}' for k,v in query.items()])
        if operator != None and operator_query != None:
            if operator.lower() == 'or' or operator.lower() == 'not':
                operator = operator.upper()
                if isinstance(operator_query, str):
                    query = f'{query} {operator} {operator_query}'
        query_params = urlencode({'q': query, 'type': search_type.lower()})
        return self.base_search(query_params)

In [6]:
spotify = SpotifyAPI(client_id, client_secret)

In [7]:
# One Sweet Day

In [8]:
One_Sweet = spotify.search({'track': 'One Sweet Day', 'artist': 'Mariah'}, search_type ='track')
One_Sweet

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=track%3AOne+Sweet+Day+artist%3AMariah&type=track&offset=0&limit=20',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4iHNK0tOyZPYnBU7nGAgpQ'},
       'href': 'https://api.spotify.com/v1/artists/4iHNK0tOyZPYnBU7nGAgpQ',
       'id': '4iHNK0tOyZPYnBU7nGAgpQ',
       'name': 'Mariah Carey',
       'type': 'artist',
       'uri': 'spotify:artist:4iHNK0tOyZPYnBU7nGAgpQ'}],
     'available_markets': ['AD',
      'AE',
      'AG',
      'AL',
      'AM',
      'AO',
      'AR',
      'AT',
      'AU',
      'AZ',
      'BA',
      'BB',
      'BD',
      'BE',
      'BF',
      'BG',
      'BH',
      'BI',
      'BJ',
      'BN',
      'BO',
      'BR',
      'BS',
      'BT',
      'BW',
      'BY',
      'BZ',
      'CA',
      'CD',
      'CG',
      'CH',
      'CI',
      'CL',
      'CM',
      'CO',
      'CR',
      'CV',
      'CW',
      'CY',
 

In [9]:
test = pd.DataFrame.from_dict(One_Sweet,orient='columns')
test

,tracks
href,https://api.spotify.com/v1/search?query=track%...
items,"[{'album': {'album_type': 'album', 'artists': ..."
limit,20
next,None
offset,0
previous,None
total,10


In [10]:
test= test.tracks.apply(pd.Series)
test

/var/folders/91/wchv5p25455f2k8078kdjvm40000gn/T/ipykernel_28272/2977931382.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  test= test.tracks.apply(pd.Series)
/var/folders/91/wchv5p25455f2k8078kdjvm40000gn/T/ipykernel_28272/2977931382.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  test= test.tracks.apply(pd.Series)


,0,1,2,3,4,5,6,7,8,9
href,https://api.spotify.com/v1/search?query=track%...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
items,"{'album': {'album_type': 'album', 'artists': [...","{'album': {'album_type': 'compilation', 'artis...","{'album': {'album_type': 'single', 'artists': ...","{'album': {'album_type': 'compilation', 'artis...","{'album': {'album_type': 'album', 'artists': [...","{'album': {'album_type': 'compilation', 'artis...","{'album': {'album_type': 'single', 'artists': ...","{'album': {'album_type': 'single', 'artists': ...","{'album': {'album_type': 'single', 'artists': ...","{'album': {'album_type': 'compilation', 'artis..."
limit,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
next,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
offset,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
previous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
total,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
test2 = pd.DataFrame(test.iloc[1])
test2

,items
0,"{'album': {'album_type': 'album', 'artists': [..."
1,"{'album': {'album_type': 'compilation', 'artis..."
2,"{'album': {'album_type': 'single', 'artists': ..."
3,"{'album': {'album_type': 'compilation', 'artis..."
4,"{'album': {'album_type': 'album', 'artists': [..."
5,"{'album': {'album_type': 'compilation', 'artis..."
6,"{'album': {'album_type': 'single', 'artists': ..."
7,"{'album': {'album_type': 'single', 'artists': ..."
8,"{'album': {'album_type': 'single', 'artists': ..."
9,"{'album': {'album_type': 'compilation', 'artis..."


In [12]:
test3 = test2['items'].apply(pd.Series)
test3

,album,artists,available_markets,disc_number,duration_ms,explicit,external_ids,external_urls,href,id,is_local,name,popularity,preview_url,track_number,type,uri
0,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,281066,False,{'isrc': 'USSM19501117'},{'spotify': 'https://open.spotify.com/track/7y...,https://api.spotify.com/v1/tracks/7ySbfLwdCwl1...,7ySbfLwdCwl1EM0zNCJZ38,False,One Sweet Day,68,https://p.scdn.co/mp3-preview/e0d52e9712427d64...,3,track,spotify:track:7ySbfLwdCwl1EM0zNCJZ38
1,"{'album_type': 'compilation', 'artists': [{'ex...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",2,280800,False,{'isrc': 'USSM19501117'},{'spotify': 'https://open.spotify.com/track/6b...,https://api.spotify.com/v1/tracks/6bfXGFwbUJZ5...,6bfXGFwbUJZ5NRQC5M3hzG,False,One Sweet Day,28,https://p.scdn.co/mp3-preview/c46c5fea30f5f398...,1,track,spotify:track:6bfXGFwbUJZ5NRQC5M3hzG
2,"{'album_type': 'single', 'artists': [{'externa...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,288733,False,{'isrc': 'USSM19501723'},{'spotify': 'https://open.spotify.com/track/58...,https://api.spotify.com/v1/tracks/58M4d6XbtFV2...,58M4d6XbtFV2n1D9kLvDIc,False,One Sweet Day - A Cappella,32,https://p.scdn.co/mp3-preview/958b43676a52d765...,3,track,spotify:track:58M4d6XbtFV2n1D9kLvDIc
3,"{'album_type': 'compilation', 'artists': [{'ex...",[{'external_urls': {'spotify': 'https://open.s...,"[AE, BH, CA, EG, IQ, JO, JP, KW, LB, LY, OM, P...",1,281253,False,{'isrc': 'USSM19501117'},{'spotify': 'https://open.spotify.com/track/7c...,https://api.spotify.com/v1/tracks/7ccrj8PDcIdL...,7ccrj8PDcIdLMxxrlWNRo8,False,One Sweet Day,21,https://p.scdn.co/mp3-preview/5d8e5d6a58a621cb...,10,track,spotify:track:7ccrj8PDcIdLMxxrlWNRo8
4,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",2,320120,False,{'isrc': 'USSM12004961'},{'spotify': 'https://open.spotify.com/track/4x...,https://api.spotify.com/v1/tracks/4xjJ1rDBohvb...,4xjJ1rDBohvbFjoBqf5X7v,False,One Sweet Day - Live at the Tokyo Dome,27,https://p.scdn.co/mp3-preview/b999fe91cb861b92...,8,track,spotify:track:4xjJ1rDBohvbFjoBqf5X7v
5,"{'album_type': 'compilation', 'artists': [{'ex...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,280773,False,{'isrc': 'USSM19501117'},{'spotify': 'https://open.spotify.com/track/4T...,https://api.spotify.com/v1/tracks/4TCTT2yHCnTe...,4TCTT2yHCnTeYDdqih93wz,False,One Sweet Day,19,https://p.scdn.co/mp3-preview/c564dedf7cb2c39a...,7,track,spotify:track:4TCTT2yHCnTeYDdqih93wz
6,"{'album_type': 'single', 'artists': [{'externa...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,291280,False,{'isrc': 'USSM19501817'},{'spotify': 'https://open.spotify.com/track/00...,https://api.spotify.com/v1/tracks/00ca0sJPtahG...,00ca0sJPtahG4rjQahQ44A,False,One Sweet Day - Chucky's Remix,21,https://p.scdn.co/mp3-preview/f3989aac0d03def2...,1,track,spotify:track:00ca0sJPtahG4rjQahQ44A
7,"{'album_type': 'single', 'artists': [{'externa...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,308720,False,{'isrc': 'USSM19501816'},{'spotify': 'https://open.spotify.com/track/2o...,https://api.spotify.com/v1/tracks/2oIT4k6FM6KH...,2oIT4k6FM6KH06merf292T,False,One Sweet Day - Live at Madison Square Garden ...,22,https://p.scdn.co/mp3-preview/77c194706f7f18ad...,4,track,spotify:track:2oIT4k6FM6KH06merf292T
8,"{'album_type': 'single', 'artists': [{'externa...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,292333,False,{'isrc': 'USSM19501726'},{'spotify': 'https://open.spotify.com/track/7l...,https:

In [13]:
test3.head(1)

,album,artists,available_markets,disc_number,duration_ms,explicit,external_ids,external_urls,href,id,is_local,name,popularity,preview_url,track_number,type,uri
0,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,281066,False,{'isrc': 'USSM19501117'},{'spotify': 'https://open.spotify.com/track/7y...,https://api.spotify.com/v1/tracks/7ySbfLwdCwl1...,7ySbfLwdCwl1EM0zNCJZ38,False,One Sweet Day,68,https://p.scdn.co/mp3-preview/e0d52e9712427d64...,3,track,spotify:track:7ySbfLwdCwl1EM0zNCJZ38
